In [17]:
from pulp import *
import numpy as np
from mpi_utils import *

In [19]:
class Mcpp_var:
     def __init__(self, m, n, l, s, D):
        self.m = m
        self.n = n
        self.l = l
        self.s = s
        self.D = D


def load_instances(num: int):
  if num < 10:
    num = "0"+str(num)
  else:
    num = str(num)
  
  file = open(f"Instances/inst{num}.dat", 'r')
  
  m = int(file.readline())
  n = int(file.readline())
  l = [int(x) for x in file.readline().split(" ") if x!= ""]
  s = [int(x) for x in file.readline().split(" ") if x!= ""]
  D = []
  for i in range(n+1):
      D.append([int(x) for x in file.readline().split(" ") if x!= "\n" if x!= ""])
  
  return Mcpp_var(m, n, l, s, D)   

In [20]:
mcpp = load_instances(1)

In [21]:
def And(model, a, b, name):
        """
        And(a,b)
        :param a: first parameter of And condition
        :param b: second parameter of And condition
        :param name: name of the And
        :return: 1 if a and b is true, false otherwise
        """
        delta = LpVariable(cat=LpInteger, name=name)
        model += delta <= a
        model += delta >= a + b - 1
        model += delta >= 0
        model += delta <= b
        return delta
def linear_prod(model, binary_var, countinuos_var, ub, name):
        """
        res = binary_var * countinuos_var
        :param binary_var: binary variable
        :param countinuos_var: countinuos variable
        :param ub: upper bound of the countinuos variable
        :param name: name of the product
        :return: the result of the product
        """
        res = LpVariable(cat=LpInteger, name=name)
        model += ub * binary_var >= res
        model += countinuos_var >= res
        model += countinuos_var - (1 - binary_var) * ub <= res
        model += res >= 0
        return res

In [22]:
# matrix m*n*n+1 where we have couriers, order and packages
X  = [[[LpVariable(name = f'X_{i}_{k}_{j}', lowBound = 0, upBound = 1, cat = LpBinary) 
        for j in range(mcpp.n+1)] for k in range(mcpp.n)] for i in range(mcpp.m)]

#distance made by each courier
dist_courier = [LpVariable(name = f'dist_cour{i}', cat = LpInteger) for i in range(mcpp.m)]

# partial distance made by each courier
dist_par  = [[LpVariable(name = f'dist{i}_{j}', cat = LpInteger) for j in range(mcpp.n + 2 )] for i in range(mcpp.m)] 

maximum = LpVariable(name=f'maximum', lowBound = 0, upBound = 10000, cat = LpInteger)

model = LpProblem(name=f'mcpp1', sense=LpMinimize)

#1. One hot encoding 
for i in range(mcpp.m):
    for k in range(mcpp.n):
        model += lpSum(X[i][k]) == 1

#2. Each element only once in the cube
for j in range(1,mcpp.n + 1):
    model += lpSum([[X[i][k][j] for k in range(mcpp.n)] for i in range(mcpp.m)]) == 1

#3. Load size constraint ( migliora con LpAffineSumExpression)
for i in range(mcpp.m):
    model += lpSum([mcpp.s[j]*X[i][k][j+1] for j in range(mcpp.n) for k in range(mcpp.n)]) <= mcpp.l[i]
 
#4. Every courier must start.
model += lpSum(X[i][0][0] for i in range(mcpp.m)) == 0

#5. Constraint that if I see a 0, all the following element for a courier must be 0.
for i in range(mcpp.m):
   ...


# 6. Distances traveled by each courier
for i in range(mcpp.m):
    model += dist_par[i][0] == lpSum([X[i][0][j] * mcpp.D[mcpp.n][j] for j in range(mcpp.n)])
    for k in range(1, mcpp.n):
        model += dist_par[i][k] == LpAffineExpression([(X[i][k - 1][j], mcpp.D[j][j2]) 
                                                       for j in range(mcpp.n) for j2 in range(mcpp.n)])

for i in range(mcpp.m):
    model += dist_courier[i] == lpSum(dist_par[i])

#7. Goal function. We want to minimize the max distance.
for i in range(mcpp.m):
    model += maximum >= lpSum(dist_par[i]) 
model += maximum
   
model.solve()


1